<a href="https://colab.research.google.com/github/CODERZZZ123/StockPred_NewsHeadlines/blob/master/pred_stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import nltk.corpus
import re
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from gensim.models import KeyedVectors
from sklearn.metrics.pairwise import euclidean_distances
!pip install textattack

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 38.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 6.1 MB/s eta 0

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
# import requests
# import time

# while True:
#     try:
#         requests.get('https://www.google.com')
#         print("Kept alive.")
#     except:
#         print("Failed to keep alive.")
#     time.sleep(600)

In [3]:
from sklearn import  set_config
set_config(display = 'diagram')

In [4]:
df_NewsDjia = pd.read_csv('News_DJIA.csv', delimiter=',')
df_RedditNews = pd.read_csv('News_reddit.csv', delimiter=',')
df_DjiaStock = pd.read_csv('DJIA_t.csv',delimiter=',')

FileNotFoundError: ignored

In [ ]:
for i in df_NewsDjia.columns :
    df_NewsDjia[i] = df_NewsDjia[i].astype(str)

In [ ]:
df_NewsDjia

In [ ]:
num_columns = df_NewsDjia.columns.shape[0]
col_names = df_NewsDjia.columns.values.tolist()
col_names = col_names[1:]
df_NewsDjia.loc[:, 'merged'] = " "
for each_col_ind in range(num_columns-1):
    df_NewsDjia.loc[:, 'merged'] =  df_NewsDjia.loc[:, 'merged'] + " " + df_NewsDjia[col_names[each_col_ind]] + " "

In [ ]:
df_finalised = df_NewsDjia[['merged']]

In [ ]:
date_numric_stopwords = "THOUSAND MILLION BILLION TRILLION HUNDRED ANNUAL ANNUALLY ANNUM YEAR YEARLY QUARTER QUARTERLY QTR MONTH MONTHLY WEEK WEEKLY DAY DAILY JANUARY FEBRUARY MARCH APRIL MAY JUNE JULY AUGUST SEPTEMBER OCTOBER NOVEMBER DECEMBER JAN FEB MAR APR MAY JUN JUL AUG SEP SEPT OCT NOV DEC MONDAY TUESDAY WEDNESDAY THURSDAY FRIDAY SATURDAY SUNDAY TWO THREE FOUR FIVE SIX SEVEN EIGHT NINE TEN ELEVEN TWELVE THIRTEEN FOURTEEN FIFTEEN SIXTEEN SEVENTEEN EIGHTEEN NINETEEN TWENTY THIRTY FORTY FIFTY SIXTY SEVENTY EIGHTY NINETY FIRST SECOND THIRD FOURTH FIFTH SIXTH SEVENTH EIGHTH NINTH TENTH II III IV V VI VII VIII IX X XI XII XIII XIV XV XVI XVII XVIII XIX XX"
date_numric_stopwords = date_numric_stopwords.lower()
date_numric_stopwords = date_numric_stopwords.split(" ")

In [ ]:

def remove_unecssary_word(sent):
    sent = sent.replace("b'",'')
    sent = sent.replace('b"','')
    sent = sent.lower()
    sent = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", sent)
    stop = stopwords.words('english')
    stop.extend(date_numric_stopwords)
    sent = " ".join([word for word in sent.split() if word not in (stop)])
    sent = " ".join([WordNetLemmatizer().lemmatize(word) for word in sent.split() ])
    return sent

In [ ]:
df_finalised['merged'] = df_finalised['merged'].apply(remove_unecssary_word)

In [ ]:
df_finalised.iloc[0][0]

In [ ]:
# from textattack.augmentation import EmbeddingAugmenter
# embed_aug = EmbeddingAugmenter()
# embed_aug.augment(df_finalised.iloc[0][0])

In [ ]:
# from textattack.augmentation import WordNetAugmenter
# wordnet_aug = WordNetAugmenter()
# wordnet_aug.augment(df_finalised.iloc[0][0])

In [ ]:
# from textattack.augmentation import EasyDataAugmenter
# eda_aug = EasyDataAugmenter()
# eda_aug.augment(df_finalised.iloc[0][0])

In [ ]:
df_finalised = pd.concat([df_finalised,df_NewsDjia['Label']],axis = 1)

In [ ]:
Positive_word = ""
with open('Positive_words.txt', 'r') as file:
    Positive_word = file.read().replace('\n', '')
Positive_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Positive_word.split(',') ])

In [ ]:
Negative_word = ""
with open('Negative_words.txt','r') as file:
    Negative_word = file.read().replace('\n','')
Negative_word = " ".join([WordNetLemmatizer().lemmatize(word) for word in Negative_word.split(',') ])

In [ ]:
Positive_word = Positive_word.split(',')
Negative_word = Negative_word.split(',')
Negative_word = Negative_word[0].split()
Positive_word = Positive_word[0].split()

In [ ]:
set_post = set(Positive_word)
set_neg = set(Negative_word)
def Count(obj):
    count_post = 1
    count_neg = 1
    obj = obj.split()
    for i in obj:
        if i in set_post:
            count_post = count_post + 1
        elif i in set_neg:
            count_neg = count_neg + 1
    return count_post/count_neg


In [ ]:
df_finalised['ratio_pos_neg'] = df_finalised['merged'].apply(Count)
X_data = df_finalised['merged']
# ratio_column = df_finalised['ratio_pos_neg']
# ratio_column = np.array(ratio_column).reshape((-1,1))

In [ ]:
Y_data = df_finalised['Label']
Y_data = Y_data.astype(int)

In [ ]:
from textattack.augmentation import EasyDataAugmenter
eda_aug = EasyDataAugmenter()
data_X = []
label = []
# open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'w').close()
# open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','w').close()
with open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/data.txt', 'a') as f1 , open('/content/drive/MyDrive/Colab Notebooks/Pred_stock_newheadline/label.txt','a') as f2:
  for i in range(450,X_data.shape[0]):
    data_X.append(X_data[i])
    label.append(Y_data[i])
    f1.write(X_data[i])
    f1.write(",")
    y = str(Y_data[i])
    f2.write(y)
    f2.write(",")
    if i%10 == 0:
      data = eda_aug.augment(X_data[i])
      for d in data:
        data_X.append(d)
        label.append(Y_data[i])
        f1.write(d)
        f1.write(",")
        f2.write(y)
        f2.write(",")

    print(i)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
Y_data = np.array(Y_data)
Y_data

In [ ]:
X_data[0]

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
data_X

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
X_data.shape

In [ ]:
x_train , x_test , y_train, y_test = X_data[:1590] , X_data[1591:] , Y_data[:1590] , Y_data[1591:]

In [ ]:
tf_idf = TfidfVectorizer(min_df=10,ngrams = (2,2))
x_train_tf = tf_idf.fit_transform(x_train)
x_test_tf = tf_idf.transform(x_test)
x_train = x_train_tf.toarray()
x_test = x_test_tf.toarray()

In [ ]:
mean_not_zero = x_train[x_train != 0].mean()
x_train[x_train!=0] = x_train[x_train!=0] -  mean_not_zero
x_test[x_test != 0 ] = x_test[x_test != 0] - mean_not_zero

In [ ]:
# ct_vect = CountVectorizer()
# x_train = ct_vect.fit_transform(x_train).toarray()
# x_test = ct_vect.transform(x_test).toarray()

In [ ]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
# pca = PCA()
# pca.fit(x_train_stand)
# cumsum = np.cumsum(pca.explained_variance_ratio_)*100
# d = [n for n in range(len(cumsum))]
# plt.figure(figsize=(10, 10))
# plt.plot(d,cumsum, color = 'red',label='cumulative explained variance')
# plt.title('Cumulative Explained Variance as a Function of the Number of Components')
# plt.ylabel('Cumulative Explained variance')
# plt.xlabel('Principal components')
# plt.axhline(y = 95, color='k', linestyle='--', label = '95% Explained Variance')
# plt.legend(loc='best')

In [ ]:
# VARIANCE EXPLAINED 90%
pca = PCA(0.90)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)
pca.explained_variance_ratio_

In [ ]:
# K = [int(i) for i in range(x_train.shape[0]//3)]
# param_grid = {
#     'n_neighbors' : K
# }
# knn = KNeighborsClassifier()
# grid_search = GridSearchCV(estimator=knn,scoring='accuracy',param_grid=param_grid,cv = 10,n_jobs = -1,verbose = 2)
# grid_search.fit(x_train,y_train)
# grid_search.best_estimator_
# knn = KNeighborsClassifier(n_neighbors=1)
# knn.fit(x_train,y_train)
# y_pred = knn.predict(x_test)
# print(accuracy_score(y_pred,y_test))
# print(classification_report(y_pred,y_test))

In [ ]:
# C = [0.01 , 0.05  , 0.08, 0.1, 0.3 , 0.6 , 0.9 , 1.2 , 1.7 , 2.5 , 2.9 , 3.7, 4.6 , 5.1 , 5.9 , 6.9 , 7.5 , 8.3 , 9.3 , 10]
# kernel = ['linear', 'poly', 'rbf', 'sigmoid']
# gamma = ['scale','auto']
# degree = [int(x) for x in range(1,5)]
# param_grid = {
#     'kernel' : kernel, vg
#     'gamma' : gamma,
#     'degree' : degree,
#     'C' : C
# }
# svc = SVC()
# grid_search = GridSearchCV(estimator=svc,scoring='accuracy',param_grid=param_grid,cv = 5,n_jobs = -1,verbose = 2)
# grid_search.fit(x_train,y_train)
# best_gird_search = grid_search.best_estimator_
# from sklearn.metrics import accuracy_score
# y_pred = best_gird_search.predict(x_test)
# print(confusion_matrix(y_test,y_pred))
# print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
# print(accuracy_score(y_train,best_gird_search.predict(x_train)))
# Score = cross_val_score(best_gird_search,X_data,Y_data,cv = 5)
# Score

In [ ]:
from sklearn.metrics import accuracy_score
model = SVC(C = 25, degree= 4, gamma =  'auto', kernel = 'linear')
model.fit(x_train,y_train)
y_pred = model.predict(x_test)
print(confusion_matrix(y_test,y_pred))
print("Accuracy for the trained data is {}".format(accuracy_score(y_train,model.predict(x_train))))
print("Accuracy Score {}".format(accuracy_score(y_test,y_pred)))
print("classification report {}".format(classification_report(y_test,y_pred)))
# Score = cross_val_score(model,X_data,Y_data,cv = 5)
# Score

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# model = Sequential()
# model.add(Dense(12,input_shape = (x_train.shape[1],),activation = 'relu'))
# model.add(Dense(8,activation = 'relu'))
# model.add(Dense(1,activation = 'sigmoid'))
# model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])
# model.fit(x_train,y_train,epochs=200,batch_size=1000)
# predictions = (model.predict(x_test) > 0.5).astype(int)
# abs(predictions.T - y_test).sum()/y_test.shape[0]


In [ ]:
# randomclassifier=RandomForestClassifier(n_estimators=1000,criterion='entropy')
# randomclassifier.fit(x_train,y_train)
# predictions = randomclassifier.predict(x_test)
# matrix=confusion_matrix(y_test,predictions)
# print(matrix)
# score=accuracy_score(y_test,predictions)
# print(score)
# report=classification_report(y_test,predictions)
# print(report)